# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [38]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [19]:
# load data from database
engine = create_engine('sqlite:///disaster_messages.db')
df = pd.read_sql_table('disaster_messages',engine)
X = df['message']
Y = df.iloc[:, 4:40]

### 2. Write a tokenization function to process your text data

In [20]:
def tokenize(text):
    tokens = word_tokenize(text.lower().strip())
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for token in tokens:
        clean_tokens.append(lemmatizer.lemmatize(token))
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [21]:
pipeline = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=10,min_samples_leaf=1,max_features='auto',n_jobs=-1)))
            ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [22]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [23]:
y_pred = pipeline.predict(X_test)

In [24]:
category_names = Y.columns

In [25]:
#for i in range(36):
#print(classification_report(Y_test, y_pred, target_names=Y_test.columns.values))
for i in range(36):
    print(Y_test.columns[i], ':')
    print(classification_report(Y_test.iloc[:,i], y_pred[:,i], target_names=category_names))

related :
                        precision    recall  f1-score   support

               related       0.62      0.35      0.45      1502
               request       0.82      0.94      0.88      5002
                 offer       0.50      0.06      0.11        50

           avg / total       0.77      0.80      0.77      6554

request :
                        precision    recall  f1-score   support

               related       0.88      0.98      0.93      5387
               request       0.82      0.40      0.54      1167

           avg / total       0.87      0.88      0.86      6554

offer :
                        precision    recall  f1-score   support

               related       1.00      1.00      1.00      6530
               request       0.00      0.00      0.00        24

           avg / total       0.99      1.00      0.99      6554

aid_related :
                        precision    recall  f1-score   support

               related       0.71      0.88      0.7

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 36
  .format(len(labels), len(target_names))


### 6. Improve your model
Use grid search to find better parameters. 

In [26]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f8762c90f28>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
               oob_score=False, random_state=None

In [29]:
parameters = {#'vect__ngram_range': ((1, 1), (1, 2)),
        #'vect__max_df': (0.5, 0.75, 1.0),
        #'vect__max_features': (None, 5000, 10000),
        #'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [50, 100],
        'clf__estimator__min_samples_split': [2, 3]}

cv = GridSearchCV(pipeline, param_grid=parameters,n_jobs=-1)

In [30]:
cv.fit(X_train, Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [50, 100], 'clf__estimator__min_samples_split': [2, 3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [31]:
print(cv.grid_scores_)
print(cv.best_params_)

[mean: 0.23339, std: 0.00040, params: {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 50}, mean: 0.23492, std: 0.00315, params: {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 100}, mean: 0.23009, std: 0.00567, params: {'clf__estimator__min_samples_split': 3, 'clf__estimator__n_estimators': 50}, mean: 0.23487, std: 0.00570, params: {'clf__estimator__min_samples_split': 3, 'clf__estimator__n_estimators': 100}]
{'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 100}


/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [36]:
y_pred = cv.predict(X_test)

for i in range(36):
    print(Y_test.columns[i], ':')
    print(classification_report(Y_test.iloc[:,i], y_pred[:,i]))

related :
             precision    recall  f1-score   support

          0       0.74      0.27      0.39      1502
          1       0.81      0.97      0.88      5002
          2       0.60      0.06      0.11        50

avg / total       0.79      0.80      0.76      6554

request :
             precision    recall  f1-score   support

          0       0.89      0.99      0.94      5387
          1       0.88      0.43      0.58      1167

avg / total       0.89      0.89      0.87      6554

offer :
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6530
          1       0.00      0.00      0.00        24

avg / total       0.99      1.00      0.99      6554

aid_related :
             precision    recall  f1-score   support

          0       0.76      0.88      0.82      3804
          1       0.79      0.62      0.69      2750

avg / total       0.77      0.77      0.76      6554

medical_help :
             precision    reca

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [39]:
with open('DisasterResponse_classifier.pkl', 'wb') as file:
    pickle.dump(cv.best_params_, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.